In [6]:
%cd project/

/home/rapids/notebooks/project


In [1]:
from skmob.utils.plot import plot_gdf
from skmob.tessellation import tilers
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from skmob.utils import utils
from math import pi, asin, sin, cos, sqrt

pd.set_option('display.max_columns', 500)



%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.offsetbox import AnchoredText
plt.rcParams["figure.figsize"] = (12, 9) # (w, h)

In [2]:
def distance_earth_km(src, dest):

    lat1, lat2 = src['lat']*pi/180, dest['lat']*pi/180
    lon1, lon2 = src['lon']*pi/180, dest['lon']*pi/180
    dlat, dlon = lat1-lat2, lon1-lon2

    ds = 2 * asin(sqrt(sin(dlat/2.0) ** 2 + cos(lat1) * cos(lat2) * sin(dlon/2.0) ** 2))
    return 6371.01 * ds

In [3]:
def filter_tessellation_land(tessellation, shape_file_land):    
    tiles_in_land = gpd.sjoin(tessellation, shape_file_land, how='left', op='intersects')
    tiles_in_land = tiles_in_land.groupby(['tile_ID'],sort=False,as_index=False).first()
    #land = tiles_in_land.dropna().drop(["index_right","boro_code","boro_name","shape_area","shape_leng"],axis=1)
    #water = tiles_in_land[tiles_in_land['index_right'].isnull()].drop(["index_right","boro_code","boro_name","shape_area","shape_leng"],axis=1)    
    
    land = tiles_in_land.dropna()[['tile_ID', 'geometry']]
    water = tiles_in_land[tiles_in_land['index_right'].isnull()][['tile_ID', 'geometry']]     
    
    crs = {'init': 'epsg:4326'}    
    land = gpd.GeoDataFrame(land, crs=crs, geometry='geometry')
    water = gpd.GeoDataFrame(water, crs=crs, geometry='geometry')     
    return {"land":land, "water":water}

In [4]:
def compute_distance_matrix(tess):

    lats_lngs = tess.geometry.apply(utils.get_geom_centroid, args=[True]).values
    
    distance_matrix = np.zeros((len(tess),len(tess)))

    for i in range(0,len(tess)):
        for j in range(0,len(tess)):
            if i != j:
                d = distance_earth_km({'lat':lats_lngs[i][0],'lon':lats_lngs[i][1]},
                                         {'lat':lats_lngs[j][0],'lon':lats_lngs[j][1]})
                distance_matrix[i,j] = d
                
    return distance_matrix

In [8]:

shape_file_land = gpd.read_file("./barriosCABA.geojson")

meters = 2612.61
tessellation = tilers.tiler.get("squared", meters=meters, base_shape=shape_file_land)
tessellation


/opt/conda/lib/python3.11/site-packages/skmob/tessellation/tilers.py:111: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  cascaded_union(polygons), crs=base_shape.crs)


,tile_ID,geometry
0,0,"POLYGON ((-58.53152 -34.68599, -58.53152 -34.6..."
1,1,"POLYGON ((-58.53152 -34.66669, -58.53152 -34.6..."
2,2,"POLYGON ((-58.53152 -34.64739, -58.53152 -34.6..."
3,3,"POLYGON ((-58.53152 -34.62808, -58.53152 -34.6..."
4,4,"POLYGON ((-58.53152 -34.60876, -58.53152 -34.5..."
...,...,...
59,59,"POLYGON ((-58.36723 -34.60876, -58.36723 -34.5..."
60,60,"POLYGON ((-58.36723 -34.58945, -58.36723 -34.5..."
61,61,"POLYGON ((-58.34376 -34.64739, -58.34376 -34.6..."
62,62,"POLYGON ((-58.34376 -34.62808, -58.34376 -34.6..."


In [10]:
plot_gdf(tessellation, style_func_args={'fillColor':'gray', 'color':'black', 'opacity': 0.2}, zoom = 9) 

In [11]:
dist = compute_distance_matrix(tessellation)
dist

array([[ 0.        ,  2.14640485,  4.29330973, ..., 17.70176717,
        18.34316055, 19.20473353],
       [ 2.14640485,  0.        ,  2.14690488, ..., 17.30889825,
        17.70589002, 18.34743174],
       [ 4.29330973,  2.14690488,  0.        , ..., 17.17723659,
        17.31292861, 17.71001181],
       ...,
       [17.70176717, 17.30889825, 17.17723659, ...,  0.        ,
         2.14740479,  4.29530935],
       [18.34316055, 17.70589002, 17.31292861, ...,  2.14740479,
         0.        ,  2.14790456],
       [19.20473353, 18.34743174, 17.71001181, ...,  4.29530935,
         2.14790456,  0.        ]])

In [12]:
np.save("./generations/sem/dist_mat.npy", dist)
np.save("./generations/dia/dist_mat.npy", dist)

In [ ]:
alls = pd.read_csv("./datasets/all_trips.csv")